#### Step 1: Rename files and make sure every chip file has an associated input file
All the data was manually reviewed and renamed for human readability. Since there was a single input sample seqenced that would serve as an input for several chipseq files, the input was duplicated such that each chipseq file has an associated input file. The data was then transfered to O2 for processing. <br>

```
srun --pty -p interactive --mem 80G -t 0-06:00 /bin/bash
gunzip *.gz
```

#### Step 2: Create a sample description file and edit to include metadata regarding the samples

```
(echo 'samplename,description'; for f in raw_files/*fq*; do readlink -f $f | perl -pe 's/(.*?_(S[0-9]+)_.*)/\1,\2/'; done) > alignment.csv
```

Edit the alignment.csv file to include, description, batch and phenotype. <br>

#### Step 3: Create a Yaml File

```
details:
- analysis: chip-seq
  genome_build: hg38
  algorithm:
    aligner: bowtie2
    peakcaller: [macs2]
  resources:
    macs2:
      options: ["-q 0.05"]
```

##### Step 4: Intiate bcBio.

```
module load bcbio/latest
unset PYTHONPATH
bcbio_nextgen.py -w template O2.yaml alignment.csv raw_files/
```

##### Step-5: Create Submission script to O2.

```
vim submit_bcbio.sh

#!/bin/sh
#SBATCH -p priority
#SBATCH -J wenchao
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 7-00:00
#SBATCH --cpus-per-task=20
#SBATCH --mem=120G
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

export PATH=/n/app/bcbio/tools/bin:$PATH
bcbio_nextgen.py ../config/alignment.yaml \
    -n 24 -t local
```

##### Step-6: Submit job to O2 for processing

```
cp submit_bcbio.sh alignment/work
cd alignment/work
sbatch submit_bcbio.sh
```

